In [5]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw



Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!


In [13]:

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        # 输入1通道，输出10通道，kernel 5*5
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.conv3 = nn.Conv2d(20, 40, 3)

        self.mp = nn.MaxPool2d(2)
        # fully connect
        self.fc = nn.Linear(40, 10)#（in_features, out_features）

    def forward(self, x):
        # in_size = 128
        in_size = x.size(0) # one batch     此时的x是包含batchsize维度为4的tensor，即(batchsize，channels，x，y)，x.size(0)指batchsize的值    把batchsize的值作为网络的in_size
        # x: 128*1*28*28
        x = F.relu(self.mp(self.conv1(x)))
        # x: 128*10*12*12  feature map =[(28-4)/2]^2=12*12
        x = F.relu(self.mp(self.conv2(x)))
        # x: 128*20*4*4
        x = F.relu(self.mp(self.conv3(x)))

        x = x.view(in_size, -1) # flatten the tensor 相当于resharp
        # print(x.size())
        # x: 128*320
        x = self.fc(x)
        # x:128*10
        # print(x.size())
        return F.log_softmax(x)  #128*10




criterion = nn.CrossEntropyLoss()    
model = SimpleNet()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [16]:
def train(epoch):
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):#batch_idx是enumerate（）函数自带的索引，从0开始
        # data.size():[128, 1, 28, 28]
        # target.size():[128]

        output = model(data)
        #output:128*10

        loss=criterion(output, target)
        #loss = F.nll_loss(output, target)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()))

        optimizer.zero_grad()   # 所有参数的梯度清零
        loss.backward()         #即反向传播求梯度
        optimizer.step()        #调用optimizer进行梯度下降更新参数
    print('\nTrain set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
        
        
        

def test():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #print(pred)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

/home/fengdahu/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.315385
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.162656
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.096972

Train set:  Accuracy: 57380/60000 (96%)



/home/fengdahu/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.1100, Accuracy: 9647/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.082535
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.056804
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.107519

Train set:  Accuracy: 57965/60000 (97%)


Test set: Average loss: 0.0884, Accuracy: 9700/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.157537
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.078290
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.051071

Train set:  Accuracy: 58298/60000 (97%)


Test set: Average loss: 0.0778, Accuracy: 9740/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.093892
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.062354
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.125021

Train set:  Accuracy: 58500/60000 (98%)


Test set: Average loss: 0.0682, Accuracy: 9767/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.157255
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.085413
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.167450

Train set:  Accuracy: 58635/60000 (98%)


Test se